In [6]:
from pymatgen.io.vasp.inputs import Incar, Kpoints, Poscar, Potcar, VaspInput
from pymatgen.core import Structure, Lattice
from pymatgen.io import vasp
import pymatgen as mg
import numpy as np
from numpy.linalg import inv
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from random import randrange
from itertools import product, combinations
from collections import defaultdict
import shutil
import os
import re
import zipfile
#import torch
def mpdata_duplicates(seq):
    resultstraintally = defaultdict(list)
    for i,item in enumerate(seq):
        resultstraintally[item].append(i)
    return ((locs) for key,locs in resultstraintally.items() 
                            if len(locs)>1)
def delete_1(a):
    delete, delete_f = [], []
    for i in range(len(a)):
        for s in a[i]:
            if s == 1:
                delete.append(i)
    for i in delete:
        if i not in delete_f:
            delete_f.append(i)
    a = np.delete(a, delete_f, axis = 0)
    return a

def supercell_gen_irre(x_axis, y_axis, z_axis):
    x_axis, y_axis, z_axis = x_axis * 2 + 1, y_axis * 2 + 1, z_axis * 2 + 1
    zero_cell = np.zeros(((x_axis, y_axis, z_axis)))
    for i in range(x_axis):
        for j in range(y_axis):
            for k in range(z_axis):
                if i%2 == 1:
                    zero_cell[i][j][k] = abs(j%2 - k%2)
                elif i%2 == 0:
                    zero_cell[i][j][k] = 1 - abs(j%2 - k%2)
    return zero_cell

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        #print("Created Directory : ", directory)
    else:
        print("Directory already existed : ", directory)
    return directory

def getfilename(path):
    zip_list = []
    f_list = os.listdir(path)
    for i in f_list:
        if os.path.splitext(i)[1] == '.zip':
            zip_list.append(i)
            print(i)
    return zip_list
#zero_cell = supercell_gen_irre(3, 4, 4)

def getfoldername(path):
    f_list = os.listdir(path)
    for i in f_list:
        if os.path.splitext(i)[1] == '':
            print(i)
    return

def supercell_gen(cell_num1):
    atom_pos = cell_num1*2 + 1
    zero_cell = np.zeros(((atom_pos, atom_pos, atom_pos)))
    for i in range(atom_pos):
        for j in range(atom_pos):
            for k in range(atom_pos):
                if i%2 == 1:
                    zero_cell[i][j][k] = abs(j%2 - k%2)
                elif i%2 == 0:
                    zero_cell[i][j][k] = 1 - abs(j%2 - k%2)
    return zero_cell

def gen_crconi(cr_content, co_content, num):
    unit = supercell_gen(num)
    ind_1f, ind_1s = [], []
    #for x in range(100000):
    ind = [i[0] for i in np.ndenumerate(unit) if i[1] == 1]
    tuple_len = [i for i in range(len(ind))]
    train_size = int(len(tuple_len)*cr_content) + randrange(2)
    co_content = co_content/(1-cr_content)
    ind1, ind23 = train_test_split(tuple_len, test_size = len(tuple_len) - train_size, train_size = train_size)
    train_size_sub = int(len(ind23)*co_content) + randrange(2)
    ind2, ind3 = train_test_split(ind23, test_size = len(ind23) - train_size_sub, train_size = train_size_sub)
    
    num = num * 2
    
    cr_position = np.array([ind[i] for i in ind1])/num
    co_position = np.array([ind[i] for i in ind2])/num
    ni_position = np.array([ind[i] for i in ind3])/num
    
    return cr_position, co_position, ni_position, ind

def bash_cp(n, file):
    srcdir = f'/home/wang/runx{n}.sh'
    dstdir = file + f'/runx{n}.sh'

    srcfile = os.path.join(srcdir)
    dstfile = os.path.join(dstdir)
    shutil.copyfile(srcfile, dstfile)

    split_pattern = '[/\\\_]'
    type_dir = re.split(split_pattern, file)
    jobname = 't'+list(type_dir[-1])[-1]+type_dir[-5]

    with open(dstdir, 'w') as rsh:
        if n == 56:
            rsh.write(f'''\
#!bin/sh
#$ -cwd
#$ -V -S /bin/bash
#$ -N {jobname}
#$ -pe smp {n}
#$ -q x64
mpirun -np {n} vasp
    ''')
        else:
            rsh.write(f'''\
#!bin/sh
#$ -cwd
#$ -V -S /bin/bash
#$ -N {jobname}
#$ -pe smp {n}
#$ -q x{n}
mpirun -np {n} vasp
    ''')
        
def write_input(output_dir, param, cell_num, kpt, n,
               cr_position, co_position, ni_position):
    incar1 = {
        'ISTART' : 0,
        'ICHARG' : 2,
        'ENCUT' : 450,
        'ALGO' : 'Normal',
        'NELM' : 60,
        'EDIFF' : 1e-07,
        'ISMEAR' : 1,
        'SIGMA' : 0.1,
        'PREC' : 'Accurate',
        'POTIM' : 0.2,
        'ISIF' : 2,
        'NSW' : 500,
        'IBRION' : 2,
        'LREAL' : 'Auto',
        'EDIFFG' : 0.001,
        'LPLANE' : '.TRUE.',
        'LSCALU' : '.FALSE.',
        'LWAVE' : '.FALSE.'
    }
    #4:8*8*4
    #3:6*6*3
    incar1 = vasp.Incar(incar1)
    true_num = (cell_num*2)**3/2
    
    specie = [{'Cr0+':1}]*len(cr_position) + [{'Co0+':1}]*len(co_position) + [{'Ni0+':1}]*len(ni_position)
    position1 = np.insert(cr_position, len(cr_position), co_position, axis = 0)
    position = np.insert(position1, len(position1), ni_position, axis = 0)
    position = position.reshape(len(position), 3)
    #print(position)
    #POSCAR
    #lattice_constant = 17.5
    param1 = param * cell_num
    lattice = Lattice.from_parameters(a = param1, b = param1, c = param1, 
                                      alpha = 90, beta = 90, gamma = 90)
    #lattice = mg.Lattice(lattice)

    struct = Structure(
        lattice,
        specie,
        position
    )
    poscar = vasp.Poscar(struct, comment = 'SQS CrCoNi')

    #POTCAR
    potcar = Potcar(symbols = ['Cr_pv_new', 'Co_pv', 'Ni_pv'], functional = 'PBE')
    
    #K-Points
    kpoints = Kpoints(num_kpts=0, style="Monkhorst-Pack", kpts=((kpt, kpt, kpt), ),
                      kpts_shift=(0, 0, 0))

    vaspinput = VaspInput(incar1, kpoints, poscar, potcar)
    vaspinput.write_input(output_dir)
    bash_cp(n, output_dir)
    return

In [10]:
path = '/home/wang/Downloads'
zip_list = getfilename(path)

In [12]:
path = '/home/wang/Downloads'
zip_list = getfilename(path)
for i in zip_list:
    zip_dir = path + '/' + i
    with zipfile.ZipFile(zip_dir) as existing_zip:
        existing_zip.extractall(path)
        
getfoldername(path)

drive-download-20220517T123420Z-001.zip
.ipynb_checkpoints
try_sqs
Yuyake_recog
Cr45Co15Ni40_try113
Cr45Co15Ni40_try105
Cr45Co15Ni40_try114
Cr45Co15Ni40_try106
Cr45Co15Ni40_try107
Cr45Co15Ni40_try102
Cr45Co15Ni40_try101
Cr45Co15Ni40_try100
Cr45Co15Ni40_try112
Cr45Co15Ni40_try108
Cr45Co15Ni40_try111
Cr45Co15Ni40_try110
Cr45Co15Ni40_try104
Cr45Co15Ni40_try109
Cr45Co15Ni40_try103
report
__pycache__


In [13]:
for i in range(100, 115):
    server_no = 52
    lattice_parameter = 3.537
    file = f'/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_{lattice_parameter}_3_try{i}'

    create_dir(file)
    print(file)

    input_dir = f'/home/wang/Downloads/Cr45Co15Ni40_try{i}'
    shutil.move(input_dir, file)

    cr_position = np.load(file+f'/Cr45Co15Ni40_try{i}/Cr45Co15Ni40_cr_chosen.npy')
    co_position = np.load(file+f'/Cr45Co15Ni40_try{i}/Cr45Co15Ni40_co_chosen.npy')
    ni_position = np.load(file+f'/Cr45Co15Ni40_try{i}/Cr45Co15Ni40_ni_chosen.npy')

    _=write_input(file, lattice_parameter, 3, 2, server_no,
                 cr_position, co_position, ni_position)
    
#     effec_dir_2 = np.load('/home/wang/MSAD/effe_directory_split/effe_dir_10.npy')
#     effec_dir_2 = np.insert(effec_dir_2, -1, file)
#     effec_dir_2 = np.unique(effec_dir_2)
#     np.save('/home/wang/MSAD/effe_directory_split/effe_dir_10.npy', effec_dir_2)

/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try100
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try101
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try102
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try103
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try104
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try105
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try106
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try107
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try108
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try109
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try110
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try111
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try112
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try113
/home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_tr

In [ ]:
for i in {100..114}
do
  cd /home/wang/MSAD/OUTPUTS/Cr45Co15Ni40/Cr45Co15Ni40_3_3.537_3_try$i
  qsub runx52.sh
done

In [143]:
file = '/home/wang/MSAD/OUTPUTS/Cr40Co30Ni30/Cr40Co30Ni30_3_518_4_try38'
_=write_input(file, 3.518, 4, 0.4, 0.3, 2, 52)

ideal:[102.4, 76.8, 76.8]
Cr, Co, Ni: [103, 77, 76]
variance: 0.35
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!


In [168]:
for i in range(28, 33):
    file = f'/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try{i}'
    print(file)
    _=write_input(file, 3.545, 3, 0.45, 0.25, 2, 52)

/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try28
ideal:[43.2, 16.2, 48.6]
Cr, Co, Ni: [43, 16, 49]
variance: 0.08
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!
/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try29
ideal:[43.2, 16.2, 48.6]
Cr, Co, Ni: [43, 17, 48]
variance: 0.35
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!
/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try30
ideal:[43.2, 16.2, 48.6]
Cr, Co, Ni: [43, 17, 48]
variance: 0.35
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!
/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try31
ideal:[43.2, 16.2, 48.6]
Cr, Co, Ni: [43, 17, 48]
variance: 0.35
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!
/home/wang/MSAD/OUTPUTS/Cr40Co15Ni45/Cr40Co15Ni45_3_545_3_try32
ideal:[43.2, 16.2, 48.6]
Cr, Co, Ni: [44, 16, 48]
variance: 0.35
BBBBB
IIIII
NNNNN
GGGGG
OOOOO!!!!!!


In [ ]:
if __name__ == '__main__':
    for i in np.linspace(1, 10, 10):
        dir_in = '/home/Wang/CrCoNi/CrCoNi_{}'.format(int(i))
        write_input(dir_in)

In [39]:
def dup_remov(a):
    sorted_idx = np.lexsort(a.T)
    sorted_a =  a[sorted_idx,:]
    row_mask = np.append([True],np.any(np.diff(sorted_a,axis=0),1))
    out = sorted_a[row_mask]
    return out
    print(len(out))

In [72]:
cr_position, co_position, ni_position, ind = gen_crconi()
cr_position = delete_1(cr_position)
co_position = delete_1(co_position)
ni_position = delete_1(ni_position)
len(ni_position)

169

In [2]:
unit6 = [[[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]],
        [[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0]],
        [[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]],
        [[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0]],
        [[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]],
        [[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0]],
        [[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]],
        [[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0]],
        [[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]],
        [[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0]],
        [[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1,0,1]]]

unit5 = [
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]]
    ]

unit4 = [
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1],[0,1,0,1,0,1,0,1,0],[1,0,1,0,1,0,1,0,1]]
    ]

unit3 = [
    [[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1]],
    [[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0]],
    [[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1],[0,1,0,1,0,1,0],[1,0,1,0,1,0,1]]
]

unit2 = [
    [[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1]],
    [[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0]],
    [[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1]],
    [[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0]],
    [[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1],[0,1,0,1,0],[1,0,1,0,1]]
]